In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 5.8MB/s 
     |████████████████████████████████| 61kB 22.7MB/s 
     |████████████████████████████████| 471kB 40.5MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536234 sha256=8645ed66bdaa330327fe2819d8f2dd5fdc6eb81a929e0bc9778da620b45bad3e
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=54b627e8579685072feaf8b6b41958d9fae6190bad8c7b1bc51f8108aa732d73
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=e57808d09f930f10fb860e1833b9be8cc5a0ec8acde30ed6345cbe838d87c621
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [ ]:
module = SourceModule("""
__global__ void add_two_array_2d(int nx, int ny, float *res, float *arr1, float *arr2){
  int x = threadIdx.x + blockDim.x * blockIdx.x;
  int y = threadIdx.y + blockDim.y * blockIdx.y;
  int ij = nx * y + x;
  if (x < nx && y < ny){
      res[ij] = arr1[ij] + arr2[ij]; 
  }
}
""")

In [ ]:
add_two_array = module.get_function("add_two_array_2d")

In [ ]:
num_x, num_y = np.int32(5), np.int32(2)
num_components = num_x * num_y

In [ ]:
arr1 = np.arange(num_components, dtype=np.float32).reshape(num_y, num_x)

In [ ]:
np.random.seed(123)
arr2 = 10 * np.random.rand(num_y, num_x)
arr2 = arr2.astype(np.float32)

In [ ]:
res = np.zeros([num_y, num_x], dtype=np.float32)

In [ ]:
arr1_gpu = gpuarray.to_gpu(arr1)
arr2_gpu = gpuarray.to_gpu(arr2)
res_gpu = gpuarray.to_gpu(res)

In [ ]:
threads_per_block = (16, 16, 1)
block_x = math.ceil(num_x / threads_per_block[0])
block_y = math.ceil(num_y / threads_per_block[1])
blocks_per_grid = (block_x, block_y, 1)

In [ ]:
add_two_array(num_x, num_y, res_gpu, arr1_gpu, arr2_gpu, block=threads_per_block, grid=blocks_per_grid)

In [ ]:
res_gpu.get()

array([[ 6.9646916,  3.8613935,  4.2685146,  8.513147 , 11.19469  ],
       [ 9.231065 , 15.807642 , 13.848297 , 12.809319 , 12.921175 ]],
      dtype=float32)

In [ ]:
arr1 + arr2

array([[ 6.9646916,  3.8613935,  4.2685146,  8.513147 , 11.19469  ],
       [ 9.231065 , 15.807642 , 13.848297 , 12.809319 , 12.921175 ]],
      dtype=float32)